In [ ]:
import pandas as pd 
import seaborn as sns
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
# Run Multicolinaryty 
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


In [ ]:
df = pd.read_csv('df_clean.csv')

In [ ]:
df.date = pd.to_datetime(df.date)
df = df.set_index('date')

# Correlation analysis 
* Some values are very correlated and others are not
* We can see that more the team as a low rank more the overall scores are low 
* When scores are high/low in some categories it tends to be high on the others two 
* We get a confirmation than the away_team_score is negatively correlate on the result 

In [ ]:
# Create the correlation matrix
corr_mean = df.corr(method = 'pearson')

#create a mask for the upper triangle
mask = np.triu(np.ones_like(corr_mean, dtype=bool))

# Add the mask to the heatmap
fig, ax = plt.subplots(figsize=(12,8)) 
ax = sns.heatmap(corr_mean, mask=mask, cmap= "RdYlGn", center=0, linewidths=1, annot=True, fmt=".2f")
#ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
#ax.set_yticklabels(ax.get_xticklabels(), rotation=45)
plt.show()

In [ ]:

sns.set_theme(style="ticks")

a = sns.pairplot(df, hue="result")

In [79]:
df = df.drop(['home_team_total_fifa_points', 'away_team_total_fifa_points','home_team_mean_defense_score' ], axis=1)
df.to_csv('df_for_ML_countries_not_numeric.csv', encoding='utf-8', index=True)

In [ ]:
# Conversion des pays en numériques
le = preprocessing.LabelEncoder()
df['home_team'] = le.fit_transform(df['home_team'])
df['away_team'] = le.fit_transform(df['away_team'])

## Vif Analysis
* It apperars to have 3 columns over Vif>5 , we should remove it 

In [ ]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant

def variance_inflation_factors(exog_df):
    '''
    Parameters
    ----------
    exog_df : dataframe, (nobs, k_vars)
        design matrix with all explanatory variables, as for example used in
        regression.

    Returns
    -------
    vif : Series
        variance inflation factors
    '''
    exog_df = add_constant(exog_df)
    vifs = pd.Series(
        [1 / (1. - OLS(exog_df[col].values, 
                       exog_df.loc[:, exog_df.columns != col].values).fit().rsquared) 
         for col in exog_df],
        index=exog_df.columns,
        name='VIF'
    )
    return vifs

variance_inflation_factors(df)

In [ ]:
df = df.drop(['home_team_total_fifa_points', 'away_team_total_fifa_points','home_team_mean_defense_score' ], axis=1)

In [ ]:
df.to_csv('df_for_ML.csv', encoding='utf-8', index=True)

In [ ]:
df